In [65]:
import pandas as pd
import numpy as np
import requests
import gc

## Address
- n_tx
- tot_rec/sent
- final_balance
- txs [ hash, fee, size, inputs[index, value], out[value] ]

values: _ , 8 cifre 

## Classes
- exchange
- dw_market
- mixer
- mining_pool
- miner
- services
- gambling

## Features

- n_transactions  
- n_in_tx   
- n_out_tx   
- tx_ratio   
- Max_in_tx_size  
- min_in_tx_size  
- avg_in_tx_size  
- Max_out_tx_size  
- min_out_tx_size  
- avg_out_tx_size  
- min_tx_interval  
- Max_tx_interval  
- avg_tx_interval  
- std_dev_tx_interval   
- tot_rec_btc  
- tot_sent_btc   
- actual_balance    
- min_sent_btc   
- Max_sent_btc   
- avg_sent_btc   
- std_dev_sent_btc  
- min_rec_btc   
- Max_rec_btc   
- avg_rec_btc   
- std_dev_rec_btc  
- min_daily_tx  
- Max_daily_tx  
- avg_daily_tx  
- std_dev_daily_tx  
- Max_fee 
- min_fee  
- avg_fee  
- std_dev_fee 
- n_reused_address_in  
- n_reused_address_out 

# Service dataset Merge

In [2]:
df = pd.read_csv('E:/Dataset/addresses-labels.csv') 
#HARVARD https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/KEWU0N

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17518402 entries, 0 to 17518401
Data columns (total 2 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   address  object
 1   label    object
dtypes: object(2)
memory usage: 267.3+ MB


In [4]:
df['label'] = df['label'].replace('coinjoin-like', 'mixer')

In [91]:
df.value_counts('label')

label
services       13510172
exchange       10220546
dw_market        347413
mixer            225445
mining_pool       78721
miner              4030
gambling            911
Name: count, dtype: int64

In [ ]:
#Entity-address dataset for 2010-2018 Bitcoin transactions
#https://github.com/Maru92/EntityAddressBitcoin?tab=readme-ov-file

## Mining

In [6]:
mining = pd.read_csv('E:/Dataset/data/Mining_full_detailed.csv')
mining = mining.drop(mining[mining['hashAdd'] == 'Nonstandard'].index)
dup = mining[mining['hashAdd'].isin(df['address'])]

In [12]:
dup.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33 entries, 0 to 69697
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   add_num        33 non-null     int64 
 1   add_type       33 non-null     int64 
 2   mining         33 non-null     object
 3   date_first_tx  33 non-null     object
 4   hashAdd        33 non-null     object
dtypes: int64(2), object(3)
memory usage: 1.5+ KB


In [13]:
mining = mining.drop(index = dup.index)

In [14]:
mining = mining.drop(columns = ['add_num', 'add_type', 'mining', 'date_first_tx'])
mining['label'] = 'mining_pool'
mining = mining.rename(columns = {'hashAdd': 'address'})

In [16]:
df = pd.concat([df,mining], ignore_index = True)

In [63]:
del mining
gc.collect()

## Service

In [18]:
service = pd.read_csv('E:/Dataset/data/Services_full_detailed.csv')
exchange = exchange.rename(columns = {'hashAdd': 'address'})

In [56]:
service = service.drop(service[service['service'] == 'StrongCoin.com-fee'].index)

In [19]:
service.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17441077 entries, 0 to 17441076
Data columns (total 5 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   add_num        int64 
 1   add_type       int64 
 2   service        object
 3   date_first_tx  object
 4   hashAdd        object
dtypes: int64(2), object(3)
memory usage: 665.3+ MB


In [57]:
service.value_counts('service')

service
ePay.info           12469529
CoinPayments.net      487768
BitPay.com            186000
CoinKite.com          115775
HaoBTC.com             33331
Purse.io               32823
OkLink.com             30183
BTCJam.com             27343
Loanbase.com           22597
FaucetBOX.com          21693
MoonBit.co.in          19437
ChangeTip.com          15905
CoinApult.com          15495
CryptoStocks.com       12486
Bitbond.com             6206
BTCPop.co               3485
CoinURL.com             2978
BitClix.com             2023
BTCLend.org             1713
Bylls.com               1314
WatchMyBit.com           614
Vic-Socks.to             125
SecureVPN.to              35
CoinWorker.com             2
Name: count, dtype: int64

In [22]:
dup = service[service['address'].isin(df['address'])]

In [23]:
service = service.drop(index = dup.index)

### Split Services

Mixers

In [24]:
mixer = service[(service['service'] == 'HelixMixer') | 
                (service['service'] == 'BitcoinFog') |
                (service['service'] == 'BitLaunder.com')]

service = service.drop(index = mixer.index)

In [25]:
mixer = mixer.drop(columns = ['add_num', 'add_type', 'service', 'date_first_tx'])
mixer['label'] = 'mixer'

In [26]:
df = pd.concat([df, mixer], ignore_index=True)
del mixer

Exchanges

In [30]:
exchange = service[(service['service'] == 'Xapo.com') |
                   (service['service'] == 'Cryptonator.com') |
                   (service['service'] == 'BitoEX.com') |
                   (service['service'] == 'BitcoinWallet.com') |
                   (service['service'] == 'HolyTransaction.com') |
                   (service['service'] == 'Paymium.com') |
                   (service['service'] == 'CoinJar.com') |
                   (service['service'] == 'CoinBox.me') |
                   (service['service'] == 'StrongCoin.com') |
                   (service['service'] == 'Cubits.com') |
                   (service['service'] == 'Cryptopay.me') |
                   (service['service'] == 'BitNZ.com') |
                   (service['service'] == 'GoCelery.com')]

service = service.drop(index = exchange.index)

In [32]:
exchange = exchange.drop(columns = ['add_num', 'add_type', 'service', 'date_first_tx'])
exchange['label'] = 'exchange'

In [36]:
df = pd.concat([df, exchange], ignore_index=True)
del exchange

Dark Web markets

In [38]:
dw_market = service[(service['service'] == 'AlphaBayMarket') | 
                    (service['service'] == 'NucleusMarket') |
                    (service['service'] == 'GermanPlazaMarket') | 
                    (service['service'] == 'DoctorDMarket') |
                    (service['service'] == 'GreenRoadMarket')]

service = service.drop(index = dw_market.index)

In [42]:
dw_market = dw_market.drop(columns = ['add_num', 'add_type', 'service', 'date_first_tx'])
dw_market['label'] = 'dw_market'

In [44]:
df = pd.concat([df, dw_market], ignore_index=True)
del dw_market

Mining Pool

In [51]:
mining = service[(service['service'] == 'Genesis-Mining.com')]

service = service.drop(index = mining.index)

In [52]:
mining = mining.drop(columns = ['add_num', 'add_type', 'service', 'date_first_tx'])
mining['label'] = 'mining_pool'

In [53]:
df = pd.concat([df, mining], ignore_index=True)
del mining

Generic services

In [58]:
service = service.drop(columns = ['add_num', 'add_type', 'service', 'date_first_tx'])
service['label'] = 'services'

In [60]:
df = pd.concat([df, service], ignore_index=True)

In [67]:
del service
gc.collect()

0

## Exchanges

In [74]:
exchange = pd.read_csv('E:/Dataset/data/Exchanges_full_detailed.csv')
exchange = exchange.drop_duplicates()
exchange = exchange.rename(columns = {'hashAdd': 'address'})

In [79]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None,'display.max_colwidth', 1000):
    print(exchange.value_counts('exchange'))

exchange
Bittrex.com              1444927
Poloniex.com              967604
CoinTrader.net            770486
LocalBitcoins.com         522287
Bitstamp.net              350556
Huobi.com                 332057
Luno.com                  309955
BTC-e.com                 307792
YoBit.net                 263108
Cex.io                    238502
Bitcoin.de                194140
HitBtc.com                187598
OKCoin.com                152423
Cryptsy.com               134559
AnxPro.com                133273
Kraken.com                114489
Bleutrade.com              87091
Hashnest.com               84857
BtcTrade.com               83539
Bter.com                   79719
BTCC.com                   64803
MercadoBitcoin.com.br      64328
MaiCoin.com                57759
VirWoX.com                 55251
Paxful.com                 55171
Cavirtex.com               53102
BX.in.th                   49005
Matbea.com                 47357
CoinSpot.com.au            45890
C-Cex.com                  35433
V

In [79]:
dup = exchange[exchange['address'].isin(df['address'])]
dup.info()

<class 'pandas.core.frame.DataFrame'>
Index: 956 entries, 20 to 7648422
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   add_num        956 non-null    int64  
 1   add_type       956 non-null    float64
 2   country        956 non-null    object 
 3   date_first_tx  956 non-null    object 
 4   exchange       956 non-null    object 
 5   address        956 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 52.3+ KB


In [80]:
exchange = exchange.drop(index = dup.index)

In [84]:
exchange = exchange.drop(exchange[(exchange['exchange'] == 'LocalBitcoins.com') | 
                                  (exchange['exchange'] == 'Hashnest.com') |
                                  (exchange['exchange'] == 'SimpleCoin.cz') |
                                  (exchange['exchange'] == 'FoxBit.com.br') |
                                  (exchange['exchange'] == 'ExchangeMyCoins.com') |
                                  (exchange['exchange'] == 'CoinArch.com') |
                                  (exchange['exchange'] == 'Coinomat.com') |
                                  (exchange['exchange'] == 'HappyCoins.com') |
                                  (exchange['exchange'] == 'CleverCoin.com') |
                                  (exchange['exchange'] == 'BitcoinVietnam.com.vn') |
                                  (exchange['exchange'] == 'Indacoin.com') |
                                  (exchange['exchange'] == 'UseCryptos.com') |
                                  (exchange['exchange'] == 'Coinimal.com') |
                                  (exchange['exchange'] == 'Coinbroker.io') |
                                  (exchange['exchange'] == 'BTradeAustralia.com') |
                                  (exchange['exchange'] == 'Exchanging.ir') |
                                  (exchange['exchange'] == 'Exchange-Credit.ru') |
                                  (exchange['exchange'] == 'Zyado.com')].index)


In [86]:
exchange = exchange.drop(columns = ['add_num', 'add_type', 'country', 'exchange', 'date_first_tx'])
exchange['label'] = 'exchange'

In [90]:
df = pd.concat([df,exchange], ignore_index = True)
del exchange

## Gambling

In [92]:
gambling = pd.read_csv('E:/Dataset/data/Gambling_full_detailed.csv')
gambling = gambling.rename(columns = {'hashAdd': 'address'})
gambling.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2665548 entries, 0 to 2665547
Data columns (total 5 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   add_num        int64 
 1   add_type       int64 
 2   gambling       object
 3   date_first_tx  object
 4   hashAdd        object
dtypes: int64(2), object(3)
memory usage: 101.7+ MB


In [93]:
gambling.value_counts('gambling')

gambling
CoinGaming.io               656699
999Dice.com                 651547
SatoshiMines.com            254601
NitrogenSports.eu           197155
PocketDice.io               125004
FortuneJack.com             120491
SecondsTrade.com             96890
BitZino.com                  78849
Rollin.io                    74602
CloudBet.com                 67795
PrimeDice.com                54640
SatoshiDice.com              41150
SatoshiBet.com               32701
SafeDice.com                 28567
SwCPoker.eu                  27693
Betcoin.ag                   25960
Coinroll.com                 22304
Crypto-Games.net             18489
AnoniBet.com                 15965
SatoshiRoulette.com          10900
BTCOracle.com                 9122
BitcoinVideoCasino.com        8032
Peerbet.org                   7766
777Coin.com                   5966
SatoshiCircle.com             4875
Coinichiwa.com                4338
Betcoin.tm                    4277
JetWin.com                    3840
BitZillions

In [96]:
dup = gambling[gambling['address'].isin(df['address'])]

In [97]:
dup.info()

<class 'pandas.core.frame.DataFrame'>
Index: 790 entries, 0 to 2665536
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   add_num        790 non-null    int64 
 1   add_type       790 non-null    int64 
 2   gambling       790 non-null    object
 3   date_first_tx  790 non-null    object
 4   address        790 non-null    object
dtypes: int64(2), object(3)
memory usage: 37.0+ KB


In [98]:
gambling = gambling.drop(index = dup.index)
gambling = gambling.drop(columns = ['add_num', 'add_type', 'date_first_tx', 'gambling'])
gambling['label'] = 'gambling'

In [100]:
df = pd.concat([df, gambling], ignore_index=True)
del gambling

In [101]:
df.value_counts('label')

label
services       13510172
exchange       10220546
gambling        2665669
dw_market        347413
mixer            225445
mining_pool       78721
miner              4030
Name: count, dtype: int64

In [103]:
df.to_csv('address_merge_tagged.csv')

In [ ]:
del df
gc.collect()